### 1. Imperative and declarative programming training

In [1]:
import math

def min_imperative(c):
    """by Ilias"""
    minValue = math.inf
    for v in c:
        if v < minValue:
            minValue = v
    return minValue

In [2]:
from functools import reduce

def min_declarative(array):
    """by Ilias"""
    return reduce(lambda a, b: a if (a < b) else b, array)

In [ ]:
# don't do that, its just for training. Ilias's desicion is much better

mymin = lambda l, m = 99999999: [(m := elem) for elem in l if elem < m][-1]

In [3]:
tests = [
    [1, 2, 3],
    [3, 2, 1],
    [1],
    [0, -8, 6],
]
for test in tests:
    print(min(test) == min_imperative(test))
    print(min(test) == min_declarative(test))
    # print(min(test) == mymin(test))  # works with Python3.8 +

True
True
True
True
True
True
True
True


### 2. smart_cache decorator that caches N most popular (by arguments usage) results

In [ ]:
@smart_cache(n=2)
def return_video(*args, **kwargs):
    return '&' * 1024 * 1024 * 1024

In [ ]:
return_video(a=1)  # calculation
return_video(a=2)  # calculation
return_video(a=1)  # using cached
return_video(a=3)  # calculation
return_video(a=1)  # using cached
return_video(a=3)  # using cached or calculation
return_video(a=2)  # using cached or calculation
return_video(a=2)  # using cached

**step 1** - creating a decorator with arguments

In [4]:
import functools


def smart_cache(cache_size=10):
    cached_arguments = []  # [(args, kwargs), ...]
    cached_arguments_usage = {}  # {<index of (args, kwargs)>: <count of its usage>}
    cached_results = {}  # {<index of (args, kwargs)>: <cached result>}

    def wrapper_helper(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f'cache_size={cache_size}')
            return func(*args, **kwargs)

        return wrapper

    return wrapper_helper

In [5]:
@smart_cache(cache_size=2)
def foo(bar):
    pass

foo(1)

cache_size=2


**step 2** - creating just a cache decorator

In [6]:
import functools


def smart_cache(cache_size=10):
    cached_arguments = []  # [(args, kwargs), ...]
    cached_results = {}  # {<index of (args, kwargs)>: <cached result>}

    def wrapper_helper(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                index = cached_arguments.index((args, kwargs))
            except ValueError:
                index = None

            if index is not None:
                return cached_results[index]

            res = func(*args, **kwargs)

            index = len(cached_arguments)
            cached_arguments.append((args, kwargs))
            cached_results[index] = res

            print(cached_arguments)
            print(cached_results)

            return res

        return wrapper

    return wrapper_helper

In [7]:
@smart_cache(cache_size=2)
def foo(bar):
    print(f'bar is {bar}')

foo(1)
foo(1)
foo(2)
foo(2)
foo(3)
foo(3)

bar is 1
[((1,), {})]
{0: None}
bar is 2
[((1,), {}), ((2,), {})]
{0: None, 1: None}
bar is 3
[((1,), {}), ((2,), {}), ((3,), {})]
{0: None, 1: None, 2: None}


**step 3** - counting statistics of arguments usage

In [8]:
import functools


def smart_cache(cache_size=10):
    cached_arguments = []  # [(args, kwargs), ...]
    cached_arguments_usage = {}  # {<index of (args, kwargs)>: <count of its usage>}
    cached_results = {}  # {<index of (args, kwargs)>: <cached result>}

    def wrapper_helper(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                index = cached_arguments.index((args, kwargs))
            except ValueError:
                index = None
            
            if index is not None:
                cached_arguments_usage[index] += 1
                return cached_results[index]

            res = func(*args, **kwargs)

            index = len(cached_arguments)
            cached_arguments.append((args, kwargs))
            cached_arguments_usage[index] = 1
            cached_results[index] = res

            print(cached_arguments)
            print(cached_arguments_usage)
            print(cached_results)

            return res

        return wrapper

    return wrapper_helper

In [9]:
@smart_cache(cache_size=2)
def foo(bar):
    print(f'bar is {bar}')

foo(1)
foo(1)
foo(2)
foo(2)
foo(3)
foo(3)

bar is 1
[((1,), {})]
{0: 1}
{0: None}
bar is 2
[((1,), {}), ((2,), {})]
{0: 2, 1: 1}
{0: None, 1: None}
bar is 3
[((1,), {}), ((2,), {}), ((3,), {})]
{0: 2, 1: 2, 2: 1}
{0: None, 1: None, 2: None}


**step 4** - use usage stats to remove not popluar cached results

In [10]:
import functools


def smart_cache(cache_size=10):
    cached_arguments = []  # [(args, kwargs), ...]
    cached_arguments_usage = {}  # {<index of (args, kwargs)>: <count of its usage>}
    cached_results = {}  # {<index of (args, kwargs)>: <cached result>}

    def wrapper_helper(func):

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                index = cached_arguments.index((args, kwargs))
            except ValueError:
                index = None
            
            if index is not None:
                cached_arguments_usage[index] += 1
                if index in cached_results:
                    return cached_results[index]

            res = func(*args, **kwargs)

            if index is None:
                index = len(cached_arguments)
                cached_arguments.append((args, kwargs))
                cached_arguments_usage[index] = 1

            cached_results[index] = res

            if len(cached_results) > cache_size:
                cached_results_indexes = list(cached_results.keys())

                index_with_smallest_usage = cached_results_indexes[0]
                smallest_usage = cached_arguments_usage[index_with_smallest_usage]
                for i in cached_results_indexes:
                    if cached_arguments_usage[i] < smallest_usage:
                        index_with_smallest_usage = i
                        smallest_usage = cached_arguments_usage[i]
                del cached_results[index_with_smallest_usage]

            print(cached_arguments)
            print(cached_arguments_usage)
            print(list(cached_results.keys()))
            print()

            return res

        return wrapper

    return wrapper_helper

In [11]:
@smart_cache(cache_size=2)
def return_video(*args, **kwargs):
    return '&' * 1024 * 1024 * 1024

In [12]:
result = return_video(1, 2, 3, a=4, b=5, c=6)
result = return_video(1, 2, 3, a=4, b=5, c=7)
result = return_video(1, 2, 3, a=4, b=5, c=8)
result = return_video(1, 2, 3, a=4, b=5, c=6)
result = return_video(1, 2, 3, a=4, b=5, c=8)
result = return_video(1, 2, 3, a=4, b=5, c={1: 2, 3: {1, 2, 3}})
result = return_video(1, 2, 3, a=4, b=5, c={1: 2, 3: {1, 2, 3}})
result = return_video(1, 2, 3, a=4, b=5, c={1: 2, 3: {1, 2, 3}})

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6})]
{0: 1}
[0]

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 7})]
{0: 1, 1: 1}
[0, 1]

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 7}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 8})]
{0: 1, 1: 1, 2: 1}
[1, 2]

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 7}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 8})]
{0: 2, 1: 1, 2: 1}
[2, 0]

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 7}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 8}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': {1: 2, 3: {1, 2, 3}}})]
{0: 2, 1: 1, 2: 2, 3: 1}
[2, 0]

[((1, 2, 3), {'a': 4, 'b': 5, 'c': 6}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 7}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': 8}), ((1, 2, 3), {'a': 4, 'b': 5, 'c': {1: 2, 3: {1, 2, 3}}})]
{0: 2, 1: 1, 2: 2, 3: 2}
[0, 3]



In [13]:
len(result)

1073741824

### Dict is unhashable even inside tuple

In [14]:
{('a', {})}

TypeError: unhashable type: 'dict'

In [15]:
{('a', ('b', ('c', {})))}  # even when it is very very deep

TypeError: unhashable type: 'dict'